In [13]:
pip install pyamaze


Note: you may need to restart the kernel to use updated packages.


In [39]:
from pyamaze import maze, agent, COLOR
from collections import deque
from time import time
import heapq
import random

# 📘 Heuristic for A*
def h(cell1, cell2):
    return abs(cell1[0] - cell2[0]) + abs(cell1[1] - cell2[1])

# 🟡 Wall Follower (simplified)
def wallFollower(m, start, end):
    return DFS(m, start, end)  # Placeholder fallback for now

# 🔵 Depth-First Search
def DFS(m, start, end):
    stack = [start]
    visited = set()
    parent = {}

    while stack:
        current = stack.pop()
        if current == end:
            break
        if current in visited:
            continue
        visited.add(current)

        r, c = current
        for d in 'NESW':
            if m.maze_map[(r, c)][d]:
                dr, dc = {'N': (-1, 0), 'E': (0, 1), 'S': (1, 0), 'W': (0, -1)}[d]
                next_cell = (r + dr, c + dc)
                if next_cell not in visited:
                    parent[next_cell] = current
                    stack.append(next_cell)

    path = []
    cell = end
    while cell != start:
        path.append(cell)
        cell = parent[cell]
    path.append(start)
    return path[::-1]

# 🔴 Breadth-First Search
def BFS(m, start, end):
    queue = deque([start])
    visited = set()
    parent = {}

    while queue:
        current = queue.popleft()
        if current == end:
            break
        if current in visited:
            continue
        visited.add(current)

        r, c = current
        for d in 'NESW':
            if m.maze_map[(r, c)][d]:
                dr, dc = {'N': (-1, 0), 'E': (0, 1), 'S': (1, 0), 'W': (0, -1)}[d]
                next_cell = (r + dr, c + dc)
                if next_cell not in visited:
                    parent[next_cell] = current
                    queue.append(next_cell)

    path = []
    cell = end
    while cell != start:
        path.append(cell)
        cell = parent[cell]
    path.append(start)
    return path[::-1]

# 🟢 A* Search
def AStar(m, start, end):
    open_set = []
    heapq.heappush(open_set, (h(start, end), 0, start))
    g_score = {start: 0}
    parent = {}

    while open_set:
        _, cost, current = heapq.heappop(open_set)

        if current == end:
            break

        r, c = current
        for d in 'NESW':
            if m.maze_map[(r, c)][d]:
                dr, dc = {'N': (-1, 0), 'E': (0, 1), 'S': (1, 0), 'W': (0, -1)}[d]
                neighbor = (r + dr, c + dc)
                temp_g = g_score[current] + 1
                if neighbor not in g_score or temp_g < g_score[neighbor]:
                    g_score[neighbor] = temp_g
                    parent[neighbor] = current
                    f_score = temp_g + h(neighbor, end)
                    heapq.heappush(open_set, (f_score, temp_g, neighbor))

    path = []
    cell = end
    while cell != start:
        path.append(cell)
        cell = parent[cell]
    path.append(start)
    return path[::-1]

# 🎯 Setup Maze
m = maze(10, 10)
m.CreateMaze(loopPercent=30)

start = (m.rows, m.cols)
end = (1, 1)

# 🎮 Timer
start_time = time()

# ⏱️ Run Algorithms
wall_path = wallFollower(m, start, end)
dfs_path = DFS(m, start, end)
bfs_path = BFS(m, start, end)
astar_path = AStar(m, start, end)

end_time = time()
elapsed = round(end_time - start_time, 2)

# 💣 Traps / 💎 Collectibles
traps = random.sample(list(m.maze_map.keys()), 5)
collectibles = random.sample([cell for cell in m.maze_map if cell not in traps], 5)

# 🧭 Mark traps and collectibles in the maze
for trap in traps:
    m.maze_map[trap]['trap'] = True
for collectible in collectibles:
    m.maze_map[collectible]['collectible'] = True

# 🕹️ Agents
a1 = agent(m, color=COLOR.yellow, footprints=True)  # Wall Follower
a2 = agent(m, color=COLOR.blue, footprints=True)    # DFS
a3 = agent(m, color=COLOR.red, footprints=True)     # BFS
a4 = agent(m, color=COLOR.green, footprints=True)   # A*

# 🧭 Trace Paths
m.tracePath({a1: wall_path})
m.tracePath({a2: dfs_path})
m.tracePath({a3: bfs_path})
m.tracePath({a4: astar_path})

# 🔢 Export stats
print("🔢 Maze Stats:")
print(f"🟡 Wall Path Length: {len(wall_path)}")
print(f"🔵 DFS Path Length: {len(dfs_path)}")
print(f"🔴 BFS Path Length: {len(bfs_path)}")
print(f"🟢 A* Path Length: {len(astar_path)}")
print(f"⏱️ Time taken: {elapsed}s")

print("\n💣 Traps at:", traps)
print("💎 Collectibles at:", collectibles)

# 🎮 Run GUI
m.run()


🔢 Maze Stats:
🟡 Wall Path Length: 41
🔵 DFS Path Length: 41
🔴 BFS Path Length: 21
🟢 A* Path Length: 21
⏱️ Time taken: 0.0s

💣 Traps at: [(5, 8), (2, 1), (3, 10), (10, 4), (1, 2)]
💎 Collectibles at: [(9, 2), (8, 7), (7, 1), (2, 2), (1, 4)]


In [ ]:
import tkinter as tk
from tkinter import messagebox
import random
import heapq

CELL_SIZE = 30
MAZE_ROWS = 15
MAZE_COLS = 20
DIRS = [(-1,0), (1,0), (0,-1), (0,1)]

class MazeGame:
    def __init__(self, master):
        self.master = master
        self.master.title("🚀 Maze Navigator: BFS, DFS, A*")

        self.frame = tk.Frame(master, bg="#f0f0f0", bd=2)
        self.frame.pack(padx=10, pady=10)

        # Canvas
        self.canvas = tk.Canvas(self.frame, width=MAZE_COLS*CELL_SIZE, height=MAZE_ROWS*CELL_SIZE, bg='white', highlightthickness=2, highlightbackground="#333")
        self.canvas.grid(row=0, column=0, columnspan=4, pady=(0, 10))

        # Control Buttons
        btn_style = {'font': ('Segoe UI', 11, 'bold'), 'padx': 10, 'pady': 5, 'width': 8, 'bg': '#4CAF50', 'fg': 'white', 'bd': 0, 'activebackground': '#45a049'}

        self.dfs_btn = tk.Button(self.frame, text="DFS", command=self.solve_dfs, **btn_style)
        self.dfs_btn.grid(row=1, column=0, padx=5)

        self.bfs_btn = tk.Button(self.frame, text="BFS", command=self.solve_bfs, **btn_style)
        self.bfs_btn.grid(row=1, column=1, padx=5)

        self.astar_btn = tk.Button(self.frame, text="A*", command=self.solve_astar, **btn_style)
        self.astar_btn.grid(row=1, column=2, padx=5)

        self.back_btn = tk.Button(self.frame, text="Reset", command=self.reset_maze, **btn_style)
        self.back_btn.grid(row=1, column=3, padx=5)

        # Status bar
        self.status = tk.Label(self.frame, text="Ready", font=('Segoe UI', 10), anchor="w", bg="#e8e8e8", fg="#333")
        self.status.grid(row=2, column=0, columnspan=4, sticky="we", pady=(10, 0))

        self.generate_valid_maze()
        self.draw_maze()

    def generate_valid_maze(self):
        while True:
            self.generate_maze()
            if self.test_solvable():
                break

    def generate_maze(self):
        self.maze = [[0 if random.random() > 0.3 else 1 for _ in range(MAZE_COLS)] for _ in range(MAZE_ROWS)]
        self.start = (0, 0)
        self.end = (MAZE_ROWS - 1, MAZE_COLS - 1)
        self.maze[0][0] = 0
        self.maze[MAZE_ROWS-1][MAZE_COLS-1] = 0

    def test_solvable(self):
        queue = [self.start]
        visited = set()
        visited.add(self.start)
        while queue:
            r, c = queue.pop(0)
            if (r, c) == self.end:
                return True
            for dr, dc in DIRS:
                nr, nc = r + dr, c + dc
                if 0 <= nr < MAZE_ROWS and 0 <= nc < MAZE_COLS and self.maze[nr][nc] == 0 and (nr, nc) not in visited:
                    visited.add((nr, nc))
                    queue.append((nr, nc))
        return False

    def draw_maze(self, path=None):
        self.canvas.delete("all")
        for r in range(MAZE_ROWS):
            for c in range(MAZE_COLS):
                x1 = c * CELL_SIZE
                y1 = r * CELL_SIZE
                x2 = x1 + CELL_SIZE
                y2 = y1 + CELL_SIZE
                color = "black" if self.maze[r][c] == 1 else "white"
                self.canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="#ccc")

        if path:
            for r, c in path:
                x1 = c * CELL_SIZE
                y1 = r * CELL_SIZE
                x2 = x1 + CELL_SIZE
                y2 = y1 + CELL_SIZE
                self.canvas.create_rectangle(x1, y1, x2, y2, fill="#ADD8E6", outline="#6495ED")

        self.draw_cell(self.start, "#4CAF50")  # green
        self.draw_cell(self.end, "#f44336")    # red

    def draw_cell(self, pos, color):
        r, c = pos
        x1 = c * CELL_SIZE
        y1 = r * CELL_SIZE
        x2 = x1 + CELL_SIZE
        y2 = y1 + CELL_SIZE
        self.canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="gray")

    def reset_maze(self):
        self.generate_valid_maze()
        self.draw_maze()
        self.status.config(text="Maze reset")

    def is_valid(self, r, c):
        return 0 <= r < MAZE_ROWS and 0 <= c < MAZE_COLS and self.maze[r][c] == 0

    def reconstruct_path(self, came_from, end):
        path = []
        while end in came_from:
            path.append(end)
            end = came_from[end]
        path.reverse()
        return path

    def solve_dfs(self):
        self.status.config(text="Solving using DFS...")
        stack = [self.start]
        came_from = {}
        visited = set()
        visited.add(self.start)

        while stack:
            current = stack.pop()
            if current == self.end:
                path = self.reconstruct_path(came_from, current)
                self.draw_maze(path)
                self.status.config(text=f"DFS: Path found with {len(path)} steps")
                return

            for dr, dc in DIRS:
                nr, nc = current[0] + dr, current[1] + dc
                neighbor = (nr, nc)
                if self.is_valid(nr, nc) and neighbor not in visited:
                    visited.add(neighbor)
                    came_from[neighbor] = current
                    stack.append(neighbor)

        self.status.config(text="DFS: No path found!")

    def solve_bfs(self):
        self.status.config(text="Solving using BFS...")
        queue = [self.start]
        came_from = {}
        visited = set()
        visited.add(self.start)

        while queue:
            current = queue.pop(0)
            if current == self.end:
                path = self.reconstruct_path(came_from, current)
                self.draw_maze(path)
                self.status.config(text=f"BFS: Path found with {len(path)} steps")
                return

            for dr, dc in DIRS:
                nr, nc = current[0] + dr, current[1] + dc
                neighbor = (nr, nc)
                if self.is_valid(nr, nc) and neighbor not in visited:
                    visited.add(neighbor)
                    came_from[neighbor] = current
                    queue.append(neighbor)

        self.status.config(text="BFS: No path found!")

    def heuristic(self, a, b):
        return abs(a[0]-b[0]) + abs(a[1]-b[1])

    def solve_astar(self):
        self.status.config(text="Solving using A*...")
        open_set = []
        heapq.heappush(open_set, (0, self.start))
        came_from = {}
        g_score = {self.start: 0}

        while open_set:
            _, current = heapq.heappop(open_set)

            if current == self.end:
                path = self.reconstruct_path(came_from, current)
                self.draw_maze(path)
                self.status.config(text=f"A*: Path found with {len(path)} steps")
                return

            for dr, dc in DIRS:
                nr, nc = current[0] + dr, current[1] + dc
                neighbor = (nr, nc)
                if self.is_valid(nr, nc):
                    tentative_g = g_score[current] + 1
                    if neighbor not in g_score or tentative_g < g_score[neighbor]:
                        came_from[neighbor] = current
                        g_score[neighbor] = tentative_g
                        f_score = tentative_g + self.heuristic(neighbor, self.end)
                        heapq.heappush(open_set, (f_score, neighbor))

        self.status.config(text="A*: No path found!")

# Run the game
root = tk.Tk()
game = MazeGame(root)
root.mainloop()
